In [ ]:
%pip install groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.8/130.8 kB 2.9 MB/s eta 0:00:00


In [ ]:
import os
from groq import Groq

client = Groq(
    api_key=os.environ.get("GROQ_API_KEY"),
)

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "Explain the importance of fast language models",
        }
    ],
    model="llama-3.3-70b-versatile",
)

print(chat_completion.choices[0].message.content)

Fast language models are crucial in today's technology-driven world, offering numerous benefits across various industries and applications. Here are some key reasons why fast language models are important:

1. **Improved User Experience**: Fast language models can process and respond to user inputs quickly, providing an instant and seamless experience. This is particularly important in real-time applications like chatbots, voice assistants, and language translation software.
2. **Increased Efficiency**: Fast language models can handle a large volume of text data, enabling them to process and analyze vast amounts of information quickly. This leads to increased efficiency in tasks like text classification, sentiment analysis, and information retrieval.
3. **Real-Time Applications**: Fast language models are essential for real-time applications like speech recognition, language translation, and sentiment analysis. They can process and respond to user inputs in a matter of milliseconds, ma

In [ ]:
class Agent:
    def __init__(self, client: Groq, system: str = "") -> None:
        self.client = client
        self.system = system
        self.messages: list = []
        if self.system is not None:
            self.messages.append({"role": "system", "content": system})

    def __call__(self, message=""):
        if message:
            self.messages.append({"role": "user", "content": message})
        result = self.execute()
        self.messages.append({"role": "assistant", "content": result})
        return result

    def execute(self):
        completion = client.chat.completions.create(
            model="llama3-70b-8192", messages=self.messages
        )
        return completion.choices[0].message.content

In [ ]:
system_prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary

get_planet_mass:
e.g. get_planet_mass: Earth
returns weight of the planet in kg

Example session:

Question: What is the mass of Earth times 2?
Thought: I need to find the mass of Earth
Action: get_planet_mass: Earth
PAUSE

You will be called again with this:

Observation: 5.972e24

Thought: I need to multiply this by 2
Action: calculate: 5.972e24 * 2
PAUSE

You will be called again with this:

Observation: 1,1944×10e25

If you have the answer, output it as the Answer.

Answer: The mass of Earth times 2 is 1,1944×10e25.

Now it's your turn:
""".strip()

def calculate(operation: str) -> float:
    return eval(operation)

def get_planet_mass(planet) -> float:
    match planet.lower():
        case "earth":
            return 5.972e24
        case "jupiter":
            return 1.898e27
        case "mars":
            return 6.39e23
        case "mercury":
            return 3.285e23
        case "neptune":
            return 1.024e26
        case "saturn":
            return 5.683e26
        case "uranus":
            return 8.681e25
        case "venus":
            return 4.867e24
        case _:
            return 0.0

In [ ]:
import re


def loop(max_iterations=10, query: str = ""):

    agent = Agent(client=client, system=system_prompt)

    tools = ["calculate", "get_planet_mass"]

    next_prompt = query

    i = 0

    while i < max_iterations:
        i += 1
        result = agent(next_prompt)
        print(result)

        if "PAUSE" in result and "Action" in result:
            action = re.findall(r"Action: ([a-z_]+): (.+)", result, re.IGNORECASE)
            chosen_tool = action[0][0]
            arg = action[0][1]

            if chosen_tool in tools:
                result_tool = eval(f"{chosen_tool}('{arg}')")
                next_prompt = f"Observation: {result_tool}"

            else:
                next_prompt = "Observation: Tool not found"

            print(next_prompt)
            continue

        if "Answer" in result:
            break


loop(query="What is the mass of Earth plus the mass of Saturn and all of that times 2?")

Thought: I need to find the mass of Earth and Saturn, then add them together and multiply the result by 2.
Thought: I need to find the mass of Earth and Saturn. 
Action: get_planet_mass: Earth
PAUSE
Observation: 5.972e+24
Thought: Now I have the mass of Earth, I need to find the mass of Saturn.
Action: get_planet_mass: Saturn
PAUSE
Observation: 5.683e+26
Thought: Now I have the mass of Earth and Saturn, I need to add them together.
Action: calculate: 5.972e24 + 5.683e26
PAUSE
Observation: 5.74272e+26
Thought: Now I have the sum of the masses, I need to multiply it by 2.
Action: calculate: 5.74272e26 * 2
PAUSE
Observation: 1.148544e+27
Thought: I have the final result.
Answer: The mass of Earth plus the mass of Saturn and all of that times 2 is 1.148544e+27.


In [ ]:
neil_tyson = Agent(client=client, system=system_prompt)

In [ ]:
result = neil_tyson("What is the mass of Earth times 5?")
print(result)

Action: get_planet_mass: Earth
PAUSE


In [ ]:
neil_tyson.menssages

In [ ]:
result = neil_tyson()
print(result)

In [ ]:
observation = get_planet_mass("Earth")
print(observation)

5.972e+24


In [ ]:
next_prompt = f"Observation: {observation}"
result = neil_tyson(next_prompt)
print(result)

Thought: I have the answer
Answer: The mass of Earth times 3.55 is 2.1141375e+25.


In [ ]:
neil_tyson.menssages

In [ ]:
result = neil_tyson()
print(result)

Thought: I have the answer
Answer: The mass of Earth times 5 is 2.986e+25.


In [ ]:
observation = calculate("3.3285e23 * 5")
print(observation)

1.66425e+24


In [ ]:
next_prompt = f"Observation: {observation}"
result = neil_tyson(next_prompt)
print(result)

Thought: This is not the answer to the original question. I need to multiply this by 5.
Action: calculate: 1.66425e+24 * 5
PAUSE


In [ ]:
neil_tyson.menssages

In [ ]:
import re

result = """
Action: calculate: 5.972e24 * 5
PAUSE
"""
action = re.findall(r"Action: ([a-z]+): (.+)", result, re.IGNORECASE)

In [ ]:
action

In [1]:
import re

def agent_loop(max_iterations, system, query):
  agent = Agent(client, system_prompt)
  tools = ['calculate', 'get_planet_mass']
  next_prompt = query
  i = 0
  while i < max_iterations:
    i += 1
    result = agent(next_prompt)
    print(result)

    if "PAUSE" in result and "Action" in result:
      action = re.findall(r"Action: ([a-z]+): (.+)", result, re.IGNORECASE)
      if action: # Check if action list is not empty
        chosen_tool = action[0][0]
        arg = action[0][1]

        if chosen_tool in tools:
          result_tool = eval(f"{chosen_tool}('{arg}')")
          next_prompt = f"Observation: {result_tool}"
        else:
          next_prompt = "Observation: Tool not foud"

      else: # Handle cases where Action is not found in the expected format
        next_prompt = "Observation: Action not found in expected format"


      print(next_prompt)
      continue

    if "Answer" in result:
      break

In [ ]:
agent_loop(max_iterations=10, system=system_prompt, query="What is the mass of the Earth plus the mass of Mercury, and all of it times 5?")

Thought: I need to find the masses of Earth and Mercury and then add them together, then multiply the result by 5.

Action: get_planet_mass: Earth
PAUSE
Observation: Action not found in expected format
Thought: I need to find the masses of Earth and Mercury and then add them together, then multiply the result by 5.

Action: get_planet_mass: Earth
PAUSE
Observation: Action not found in expected format
Thought: I need to find the masses of Earth and Mercury and then add them together, then multiply the result by 5.

Action: get_planet_mass: Earth
PAUSE
Observation: Action not found in expected format
Thought: I need to find the masses of Earth and Mercury.

Action: get_planet_mass: Earth
PAUSE
Observation: Action not found in expected format
Thought: I need to find the masses of Earth and Mercury.

Action: get_planet_mass: Earth
PAUSE
Observation: Action not found in expected format
Thought: I need to find the masses of Earth and Mercury.

Action: get_planet_mass: Earth
PAUSE
Observation